<b> <font size =5> Calculate Distance from Each Facility to Monitoring Locations - Facilities with 'High' Emissions </b></font>

This iPython Notebook calculates the distance from each facility obtained from the National Emissions Inventory located in Oakland to the monitoring data points from the EDF dataset. Distances for facilities that were categorized as "high" emitting facilities are estimated here. Facilities of the same category located close to each other are combined into one single point based on their centroid to avoid multi-colinearity issues. Grouping of facilities was done in ArcGIS.

<b> Two Input Files: </b>
    
     PM_Facilities_High_Centroid.csv 
     NO2_Facilities_High_Centroid.csv


In [3]:
#Import basic python packages for data analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from shapely.ops import nearest_points

import seaborn as sns

from mpl_toolkits.axes_grid1 import make_axes_locatable

import math

import time

from matplotlib import cm

import matplotlib.lines as mlines

%matplotlib inline

# <b> <font size = 5> Load Air Pollution Monitoring Data </b> </font>

In [4]:
df = pd.read_csv('EDF_Data.csv', header = 1)
df.tail()

,Longitude,Latitude,NO Value,NO2 Value,BC Value
21483,-122.034943,37.560076,129.999995,44.778220,3.923761
21484,-122.034724,37.560164,60.799998,39.027545,1.408693
21485,-122.034681,37.559830,34.622951,28.816797,2.659885
21486,-122.034504,37.559958,74.764705,35.735434,1.776353
21487,-122.034503,37.559957,78.754782,41.062757,2.014664


## <b> <font size = 4> Split dataset into BC and NO2 </b> </font>

In [5]:
BC_df = df[['Longitude', 'Latitude', 'BC Value']]

In [6]:
NO2_df = df[['Longitude', 'Latitude', 'NO2 Value']]

# <b> <font size = 5> Load  Facility Level Data </b> </font>

In [7]:
Facility_PM = pd.read_csv("Data/PM_Facilities_High_Centroid.csv")

In [8]:
Facility_PM.head()

,latitude,longitude,source_typ,ems_type,source_cat,Grouping,EIS,Centroid_X,Centroid_Y
0,37.695818,-122.164205,FoodPlant,high,Food-Products-Processing-Plant,0,15756011,37.695818,-122.164205
1,37.709020,-122.141910,FoodPlant,high,Food-Products-Processing-Plant,14,340611,37.704465,-122.140230
2,37.705560,-122.152090,Manufacturing,high,Manufacturing,0,10708111,37.705560,-122.152090
3,37.704590,-122.174040,Manufacturing,high,Manufacturing,8,445511,37.706633,-122.179205
4,37.704586,-122.161181,Retail-Res,high,Wholesale,2,17244511,37.708139,-122.175454


In [9]:
Facility_NO2 = pd.read_csv("Data/NO2_Facilities_High_Centroid.csv")

In [10]:
Facility_NO2.head()

,latitude,longitude,source-typ,ems-type,source_cat,eis facili,Grouping,Centroid_X,Centroid_Y
0,37.69710,-122.17306,FoodPlant,high,Food-Products-Processing-Plant,804911,17,37.700845,-122.173550
1,37.70459,-122.17404,Manufacturing,high,Manufacturing,445511,8,37.706805,-122.157975
2,37.70902,-122.14191,FoodPlant,high,Food-Products-Processing-Plant,340611,14,37.707945,-122.159205
3,37.70687,-122.17650,Retail-Res,high,Wholesale,18135811,2,37.707750,-122.182175
4,37.70863,-122.18785,Manufacturing,high,Manufacturing,14071611,18,37.710605,-122.189250


# <b> <font size = 5>  Calculate distance from monitoring location to each facility for PM </b> </font>

## <b> <font size = 4> Create a new dataframe to store the distance values </b> </font>

In [11]:
#Make a copy of the Facility_PM dataframe
Facility_PM_All = Facility_PM.copy()

In [12]:
Facility_PM.head()

,latitude,longitude,source_typ,ems_type,source_cat,Grouping,EIS,Centroid_X,Centroid_Y
0,37.695818,-122.164205,FoodPlant,high,Food-Products-Processing-Plant,0,15756011,37.695818,-122.164205
1,37.709020,-122.141910,FoodPlant,high,Food-Products-Processing-Plant,14,340611,37.704465,-122.140230
2,37.705560,-122.152090,Manufacturing,high,Manufacturing,0,10708111,37.705560,-122.152090
3,37.704590,-122.174040,Manufacturing,high,Manufacturing,8,445511,37.706633,-122.179205
4,37.704586,-122.161181,Retail-Res,high,Wholesale,2,17244511,37.708139,-122.175454


In [13]:
### Create a column as eis-source-type by combining the source_type and EIS ID for each facility
Facility_PM['eis-source'] =    Facility_PM['source_typ']    + '-' +  Facility_PM['EIS'].apply(str) 

In [14]:
### Create a columns a eis-source-ems-type by combining emissions type with eis-source
Facility_PM['eis-source-ems'] =  Facility_PM['ems_type']     + '-' + Facility_PM['eis-source']

In [15]:
Facility_PM.head(100)

,latitude,longitude,source_typ,ems_type,source_cat,Grouping,EIS,Centroid_X,Centroid_Y,eis-source,eis-source-ems
0,37.695818,-122.164205,FoodPlant,high,Food-Products-Processing-Plant,0,15756011,37.695818,-122.164205,FoodPlant-15756011,high-FoodPlant-15756011
1,37.709020,-122.141910,FoodPlant,high,Food-Products-Processing-Plant,14,340611,37.704465,-122.140230,FoodPlant-340611,high-FoodPlant-340611
2,37.705560,-122.152090,Manufacturing,high,Manufacturing,0,10708111,37.705560,-122.152090,Manufacturing-10708111,high-Manufacturing-10708111
3,37.704590,-122.174040,Manufacturing,high,Manufacturing,8,445511,37.706633,-122.179205,Manufacturing-445511,high-Manufacturing-445511
4,37.704586,-122.161181,Retail-Res,high,Wholesale,2,17244511,37.708139,-122.175454,Retail-Res-17244511,high-Retail-Res-17244511
...,...,...,...,...,...,...,...,...,...,...,...
56,37.877360,-122.301290,ConcretePlant,high,Concrete-Batch-Plant,0,477811,37.877360,-122.301290,ConcretePlant-477811,high-ConcretePlant-477811
57,37.877430,-122.250880,Retail-Res,high,Institution,0,448011,37.877430,-122.250880,Retail-Res-448011,high-Retail-Res-448011
58,37.878390,-122.303300,AutoRepair,high,Auto-Body-Painting-or-Repair-Shop,0,138911,37.878390,-122.303300,AutoRepair-138911,high-AutoRepair-138911
59,37.877850,-122.305270,Retail-Res,high,Institution,7,203711,37.878429,-122.302847,Retail-Res-203711,high-Retail-Res-203711


In [16]:
### Add an empty column for distance
Facility_PM['dist'] = 0
Facility_PM['dist'].astype(float)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
     ... 
56    0.0
57    0.0
58    0.0
59    0.0
60    0.0
Name: dist, Length: 61, dtype: float64

**Next, create a new dataframe with a set of columns for each industrial facility. The set of columns for each industrial facility will include one column for latitude, longitude and PM2.5 emissions value. The PM10 emissions column is dropped since Black Carbon tracks better with PM2.5 emissions**

In [17]:
# Create individual dataframes containing the latitude, longitude and distance
Oak_PM_lat = Facility_PM[['eis-source-ems', 'Centroid_X']]
Oak_PM_long = Facility_PM[['eis-source-ems', 'Centroid_Y']]
Oak_PM_dist = Facility_PM[['eis-source-ems', 'dist']]


In [18]:
# Transpose all the dataframes
Oak_PM_lat = Oak_PM_lat.T
Oak_PM_long = Oak_PM_long.T
Oak_PM_dist = Oak_PM_dist.T

In [20]:
## Make the header as the first row in each transposed dataframe. The header will be the 'eis-source-ems' row
Oak_PM_lat = Oak_PM_lat.rename(columns=Oak_PM_lat.iloc[0]).drop(Oak_PM_lat.index[0])
Oak_PM_long = Oak_PM_long.rename(columns=Oak_PM_long.iloc[0]).drop(Oak_PM_long.index[0])
Oak_PM_dist = Oak_PM_dist.rename(columns=Oak_PM_dist.iloc[0]).drop(Oak_PM_dist.index[0])


In [21]:
## Add suffix to column header based on the dataframe type
Oak_PM_lat.columns = [str(col) + '_latitude' for col in Oak_PM_lat.columns]
Oak_PM_long.columns = [str(col) + '_longitude' for col in Oak_PM_long.columns]
Oak_PM_dist.columns = [str(col) + '_dist' for col in Oak_PM_dist.columns]


In [22]:
## Remove index for each dataframe
Oak_PM_lat.reset_index(drop=True, inplace=True)
Oak_PM_long.reset_index(drop=True, inplace=True)
Oak_PM_dist.reset_index(drop=True, inplace=True)


In [23]:
### Combine individual dataframes into one
Oak_PM_combined = Oak_PM_lat.join(Oak_PM_long).join(Oak_PM_dist)

In [24]:
### Sort based on column names
Oak_PM_combined = Oak_PM_combined.reindex(columns=sorted(Oak_PM_combined.columns))

In [25]:
Oak_PM_combined

,high-AsphaltPlant-10510811_dist,high-AsphaltPlant-10510811_latitude,high-AsphaltPlant-10510811_longitude,high-AsphaltPlant-808611_dist,high-AsphaltPlant-808611_latitude,high-AsphaltPlant-808611_longitude,high-AutoRepair-138911_dist,high-AutoRepair-138911_latitude,high-AutoRepair-138911_longitude,high-AutoRepair-15714511_dist,...,high-WWTP-379511_longitude,high-Waste-10708011_dist,high-Waste-10708011_latitude,high-Waste-10708011_longitude,high-Waste-168411_dist,high-Waste-168411_latitude,high-Waste-168411_longitude,high-Waste-18782611_dist,high-Waste-18782611_latitude,high-Waste-18782611_longitude
0,0,37.7627,-122.222,0,37.8698,-122.302,0,37.8784,-122.303,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208


In [26]:
#Create a datafram where each row contains emissions of PM2.5 for each facility
Oak_PM_combined = Oak_PM_combined.loc[Oak_PM_combined.index.repeat(21488)].reset_index(drop=True)

In [27]:
combined_BC_Facility = BC_df.join(Oak_PM_combined)

In [28]:
combined_BC_Facility.head()

,Longitude,Latitude,BC Value,high-AsphaltPlant-10510811_dist,high-AsphaltPlant-10510811_latitude,high-AsphaltPlant-10510811_longitude,high-AsphaltPlant-808611_dist,high-AsphaltPlant-808611_latitude,high-AsphaltPlant-808611_longitude,high-AutoRepair-138911_dist,...,high-WWTP-379511_longitude,high-Waste-10708011_dist,high-Waste-10708011_latitude,high-Waste-10708011_longitude,high-Waste-168411_dist,high-Waste-168411_latitude,high-Waste-168411_longitude,high-Waste-18782611_dist,high-Waste-18782611_latitude,high-Waste-18782611_longitude
0,-122.322594,37.806781,0.818032,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208
1,-122.322310,37.806150,0.551475,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208
2,-122.322301,37.806420,0.593712,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208
3,-122.322299,37.805880,0.489898,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208
4,-122.322267,37.806689,0.739341,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.297,0,37.7146,-122.19,0,37.8146,-122.293,0,37.76,-122.208


In [26]:
# Convert distance or emissions distance column to float type
for idx, col in enumerate(combined_BC_Facility.columns):
        if "_dist" in col:
            combined_BC_Facility[col] = pd.to_numeric(combined_BC_Facility[col], downcast="float")



## <b> <font size = 4>  Calculate distance between point of measurement and each facility and add it to the '_dist' column </b> </font>

In [27]:
### Defining a function to calculate the distance between two GPS coordinates (latitude and longitude)
def distance(origin, destination):
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d


In [28]:
time1 = time.time()
for index, row in combined_BC_Facility.iterrows():
    for idx, col in enumerate(combined_BC_Facility.columns):
        if "_dist" in col:
            combined_BC_Facility.at[index,col] = float(distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2])))*0.621
            #BC_Facility.at[index,col] = float(row.iloc[idx])
time2 = time.time()            
    
print(time2 - time1)

49.647422075271606


## <b> <font size = 4>  Write to a CSV file </b> </font>

##### Write this to a dataframe
combined_BC_Facility.to_csv("Data/BC_PM_Facilities_High_Dist.csv")



# <b> <font size = 5>  Calculate distance from monitoring location to each facility for NO2 </b> </font>

## <b> <font size = 4> Create a new dataframe to store the distance values </b> </font>

In [31]:
#Make a copy of the Facility_NO2 dataframe
Facility_NO2_All = Facility_NO2.copy()

In [32]:
Facility_NO2.head()

,latitude,longitude,source-typ,ems-type,source_cat,eis facili,Grouping,Centroid_X,Centroid_Y
0,37.69710,-122.17306,FoodPlant,high,Food-Products-Processing-Plant,804911,17,37.700845,-122.173550
1,37.70459,-122.17404,Manufacturing,high,Manufacturing,445511,8,37.706805,-122.157975
2,37.70902,-122.14191,FoodPlant,high,Food-Products-Processing-Plant,340611,14,37.707945,-122.159205
3,37.70687,-122.17650,Retail-Res,high,Wholesale,18135811,2,37.707750,-122.182175
4,37.70863,-122.18785,Manufacturing,high,Manufacturing,14071611,18,37.710605,-122.189250


In [33]:
### Create a column as eis-source-type by combining the source_type and EIS ID for each facility
Facility_NO2['eis-source'] =   Facility_NO2['source-typ'] + '-' +   Facility_NO2['eis facili'].apply(str) 

In [34]:
### Create a columns a eis-source-ems-type by combining emissions type with eis-source
Facility_NO2['eis-source-ems'] =  Facility_NO2['ems-type']   + '-' +  Facility_NO2['eis-source'] 

In [35]:
### Add an empty column for distance
Facility_NO2['dist'] = 0
Facility_NO2['dist'].astype(float)

0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
5     0.0
6     0.0
7     0.0
8     0.0
9     0.0
10    0.0
11    0.0
12    0.0
13    0.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    0.0
22    0.0
23    0.0
24    0.0
25    0.0
26    0.0
27    0.0
28    0.0
29    0.0
30    0.0
31    0.0
32    0.0
33    0.0
34    0.0
35    0.0
36    0.0
37    0.0
38    0.0
39    0.0
40    0.0
41    0.0
42    0.0
43    0.0
44    0.0
45    0.0
46    0.0
47    0.0
48    0.0
49    0.0
50    0.0
51    0.0
Name: dist, dtype: float64

In [36]:
Facility_NO2.head(100)

,latitude,longitude,source-typ,ems-type,source_cat,eis facili,Grouping,Centroid_X,Centroid_Y,eis-source,eis-source-ems,dist
0,37.697100,-122.173060,FoodPlant,high,Food-Products-Processing-Plant,804911,17,37.700845,-122.173550,FoodPlant-804911,high-FoodPlant-804911,0
1,37.704590,-122.174040,Manufacturing,high,Manufacturing,445511,8,37.706805,-122.157975,Manufacturing-445511,high-Manufacturing-445511,0
2,37.709020,-122.141910,FoodPlant,high,Food-Products-Processing-Plant,340611,14,37.707945,-122.159205,FoodPlant-340611,high-FoodPlant-340611,0
3,37.706870,-122.176500,Retail-Res,high,Wholesale,18135811,2,37.707750,-122.182175,Retail-Res-18135811,high-Retail-Res-18135811,0
4,37.708630,-122.187850,Manufacturing,high,Manufacturing,14071611,18,37.710605,-122.189250,Manufacturing-14071611,high-Manufacturing-14071611,0
5,37.712580,-122.190650,WWTP,high,Wastewater-Treatment-Facility,203811,0,37.712580,-122.190650,WWTP-203811,high-WWTP-203811,0
6,37.716568,-122.131500,Retail-Res,high,Institution,15721111,1,37.717313,-122.170820,Retail-Res-15721111,high-Retail-Res-15721111,0
7,37.714630,-122.190430,Waste,high,Landfill,10708011,0,37.714630,-122.190430,Waste-10708011,high-Waste-10708011,0
8,37.720740,-122.190530,Manufacturing,high,Metals,3963411,0,37.720740,-122.190530,Manufacturing-3963411,high-Manufacturing-3963411,0
9,37.722040,-122.200100,Transportation,high,Port,444911,0,37.722040,-122.200100,Transportation-444911,high-Transportation-444911,0


**Next, create a new dataframe with a set of column for each industrial facility. The set of columns for each industrial facility will include one column for latitude, longitude and NO2 emissions value.**

In [37]:
# Create individual dataframes
Oak_NO2_lat = Facility_NO2[['eis-source-ems', 'Centroid_X']]
Oak_NO2_long = Facility_NO2[['eis-source-ems', 'Centroid_Y']]
Oak_NO2_dist = Facility_NO2[['eis-source-ems', 'dist']]

In [38]:
# Transpose all the dataframes
Oak_NO2_lat = Oak_NO2_lat.T
Oak_NO2_long = Oak_NO2_long.T
Oak_NO2_dist = Oak_NO2_dist.T

In [39]:
## Make the header as the first row in each transposed dataframe
Oak_NO2_lat = Oak_NO2_lat.rename(columns=Oak_NO2_lat.iloc[0]).drop(Oak_NO2_lat.index[0])
Oak_NO2_long = Oak_NO2_long.rename(columns=Oak_NO2_long.iloc[0]).drop(Oak_NO2_long.index[0])
Oak_NO2_dist = Oak_NO2_dist.rename(columns=Oak_NO2_dist.iloc[0]).drop(Oak_NO2_dist.index[0])

In [40]:
## Add suffix to column header based on the dataframe type
Oak_NO2_lat.columns = [str(col) + '_latitude' for col in Oak_NO2_lat.columns]
Oak_NO2_long.columns = [str(col) + '_longitude' for col in Oak_NO2_long.columns]
Oak_NO2_dist.columns = [str(col) + '_dist' for col in Oak_NO2_dist.columns]

In [41]:
## Remove index for each dataframe
Oak_NO2_lat.reset_index(drop=True, inplace=True)
Oak_NO2_long.reset_index(drop=True, inplace=True)
Oak_NO2_dist.reset_index(drop=True, inplace=True)

In [42]:
### Combine individual dataframes into one
Oak_NO2_combined = Oak_NO2_lat.join(Oak_NO2_long).join(Oak_NO2_dist)

In [43]:
### Sort based on column names
Oak_NO2_combined = Oak_NO2_combined.reindex(columns=sorted(Oak_NO2_combined.columns))

In [44]:
Oak_NO2_combined

,high-AsphaltPlant-10510811_dist,high-AsphaltPlant-10510811_latitude,high-AsphaltPlant-10510811_longitude,high-AsphaltPlant-808611_dist,high-AsphaltPlant-808611_latitude,high-AsphaltPlant-808611_longitude,high-AutoRepair-195711_dist,high-AutoRepair-195711_latitude,high-AutoRepair-195711_longitude,high-EGen-10707511_dist,...,high-Transportation-444911_longitude,high-WWTP-203811_dist,high-WWTP-203811_latitude,high-WWTP-203811_longitude,high-WWTP-379511_dist,high-WWTP-379511_latitude,high-WWTP-379511_longitude,high-Waste-10708011_dist,high-Waste-10708011_latitude,high-Waste-10708011_longitude
0,0,37.7627,-122.222,0,37.8698,-122.302,0,37.8564,-122.291,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19


In [45]:
#Create a datafram where each row contains emissions of PM2.5 for each facility
Oak_NO2_combined = Oak_NO2_combined.loc[Oak_NO2_combined.index.repeat(21488)].reset_index(drop=True)

In [46]:
combined_NO2_Facility = NO2_df.join(Oak_NO2_combined)

In [47]:
combined_NO2_Facility.head()

,Longitude,Latitude,NO2 Value,high-AsphaltPlant-10510811_dist,high-AsphaltPlant-10510811_latitude,high-AsphaltPlant-10510811_longitude,high-AsphaltPlant-808611_dist,high-AsphaltPlant-808611_latitude,high-AsphaltPlant-808611_longitude,high-AutoRepair-195711_dist,...,high-Transportation-444911_longitude,high-WWTP-203811_dist,high-WWTP-203811_latitude,high-WWTP-203811_longitude,high-WWTP-379511_dist,high-WWTP-379511_latitude,high-WWTP-379511_longitude,high-Waste-10708011_dist,high-Waste-10708011_latitude,high-Waste-10708011_longitude
0,-122.322594,37.806781,17.539762,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19
1,-122.322310,37.806150,19.956750,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19
2,-122.322301,37.806420,23.967768,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19
3,-122.322299,37.805880,18.435184,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19
4,-122.322267,37.806689,25.797037,0,37.7627,-122.222,0,37.8698,-122.302,0,...,-122.2,0,37.7126,-122.191,0,37.8256,-122.297,0,37.7146,-122.19


In [48]:
# Convert distance or emissions distance column to float type
for idx, col in enumerate(combined_NO2_Facility.columns):
        if "_dist" in col:
            combined_NO2_Facility[col] = pd.to_numeric(combined_NO2_Facility[col], downcast="float")



## <b> <font size = 4>  Calculate distance between point of measurement and each facility and add it to the '_dist' column </b> </font>

In [49]:
time1 = time.time()
for index, row in combined_NO2_Facility.iterrows():
    for idx, col in enumerate(combined_NO2_Facility.columns):
        if "_dist" in col:
            combined_NO2_Facility.at[index,col] = float(distance((row.iloc[1], row.iloc[0]), (row.iloc[idx+1], row.iloc[idx+2])))*0.621
            #BC_Facility.at[index,col] = float(row.iloc[idx])
time2 = time.time()            
    
print(time2 - time1)

41.93308997154236


## <b> <font size = 4>  Write to a CSV file </b> </font>

##### Write this to a dataframe
combined_NO2_Facility.to_csv("Data/NO2_NO2_Facilities_High_Dist.csv")

